# System Integration

> System-wide keyboard hooks and integration with the improved composition engine

In [ ]:
#| default_exp system

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from typing import Dict, List, Optional, Callable
from dataclasses import dataclass
from enum import Enum
import sys
import os
import argparse
import time
from pathlib import Path

from gzkeyboard.core import CharacterStore
from gzkeyboard.composition import SyllableComposer, CompositionStrategy, CompositionResult

## Input Modes and Configuration

Define the input modes and basic configuration system, improved from the original implementation.

In [ ]:
#| export
class InputMode(Enum):
    """Defines the input modes available in the Geez keyboard."""
    TIGRINYA = "tigrinya"
    AMHARIC = "amharic"
    LATIN = "latin"  # For regular typing without conversion

@dataclass
class KeyboardConfig:
    """Configuration for the Geez keyboard input method."""
    default_mode: InputMode = InputMode.TIGRINYA
    composition_strategy: CompositionStrategy = CompositionStrategy.ADAPTIVE
    base_timeout: float = 0.5
    custom_mappings_path: Optional[str] = None
    
    # Hotkey configuration
    toggle_hotkey: str = "ctrl+space"
    tigrinya_hotkey: str = "alt+t"
    amharic_hotkey: str = "alt+a"
    latin_hotkey: str = "alt+l"
    cancel_hotkey: str = "escape"
    
    # Advanced settings
    show_notifications: bool = True
    debug_mode: bool = False

## Hotkey Manager

Manages keyboard shortcuts for controlling the input method.

In [ ]:
#| export
class HotkeyManager:
    """Manages hotkeys for controlling the Geez keyboard."""
    
    def __init__(self, config: KeyboardConfig):
        self.config = config
        self.callbacks: Dict[str, Callable] = {}
        self.registered_hotkeys: List[str] = []
    
    def register_callback(self, action: str, callback: Callable):
        """Register a callback for a specific action."""
        self.callbacks[action] = callback
    
    def setup_hotkeys(self):
        """Set up system hotkeys using the keyboard library."""
        try:
            import keyboard
            
            # Define hotkey mappings
            hotkey_map = {
                self.config.toggle_hotkey: 'toggle_input',
                self.config.tigrinya_hotkey: 'switch_to_tigrinya',
                self.config.amharic_hotkey: 'switch_to_amharic',
                self.config.latin_hotkey: 'switch_to_latin',
                self.config.cancel_hotkey: 'cancel_buffer'
            }
            
            # Register hotkeys
            for hotkey, action in hotkey_map.items():
                if action in self.callbacks:
                    keyboard.add_hotkey(hotkey, self.callbacks[action])
                    self.registered_hotkeys.append(hotkey)
                    if self.config.debug_mode:
                        print(f"Registered hotkey: {hotkey} → {action}")
            
            return True
            
        except ImportError:
            print("Keyboard library not available. Hotkeys disabled.")
            return False
    
    def cleanup(self):
        """Clean up registered hotkeys."""
        try:
            import keyboard
            for hotkey in self.registered_hotkeys:
                keyboard.remove_hotkey(hotkey)
            self.registered_hotkeys.clear()
        except ImportError:
            pass

## Status Indicator

Provides visual feedback about the current keyboard state.

In [ ]:
#| export
class StatusIndicator:
    """A status indicator for the Geez keyboard."""
    
    def __init__(self, config: KeyboardConfig):
        self.config = config
        self.visible = False
        self.current_mode = config.default_mode
        self.active = True
    
    def show_status(self, message: str, duration: float = 2.0):
        """Show a status message."""
        if self.config.show_notifications:
            print(f"[GzKeyboard] {message}")
            self.visible = True
    
    def update_mode(self, mode: InputMode):
        """Update the current input mode."""
        self.current_mode = mode
        self.show_status(f"Switched to {mode.value} mode")
    
    def toggle_active(self):
        """Toggle active/inactive state."""
        self.active = not self.active
        status = "activated" if self.active else "deactivated"
        self.show_status(f"GzKeyboard {status}")
    
    def show_buffer_state(self, buffer: str, suggestions: List[str] = None):
        """Show current buffer state for debugging."""
        if self.config.debug_mode and buffer:
            msg = f"Buffer: '{buffer}'"
            if suggestions:
                msg += f" | Suggestions: {suggestions[:3]}"
            print(f"[DEBUG] {msg}")
    
    def hide(self):
        """Hide the status indicator."""
        self.visible = False

## System Input Handler

The main class that integrates everything together and handles system-wide keyboard input.

In [ ]:
#| export
class SystemInputHandler:
    """Handles system-wide keyboard input for the Geez keyboard."""
    
    def __init__(self, config: Optional[KeyboardConfig] = None):
        """Initialize the input handler with configuration."""
        self.config = config or KeyboardConfig()
        
        # Initialize components
        self.character_store = CharacterStore()
        self.composer = SyllableComposer(
            self.character_store,
            strategy=self.config.composition_strategy,
            timeout=self.config.base_timeout
        )
        self.status = StatusIndicator(self.config)
        self.hotkey_manager = HotkeyManager(self.config)
        
        # State
        self.current_mode = self.config.default_mode
        self.active = True
        self.keyboard_hook_active = False
        
        # Setup hotkey callbacks
        self._setup_hotkey_callbacks()
    
    def _setup_hotkey_callbacks(self):
        """Set up hotkey callback functions."""
        self.hotkey_manager.register_callback('toggle_input', self.toggle_active)
        self.hotkey_manager.register_callback('switch_to_tigrinya', lambda: self.switch_mode(InputMode.TIGRINYA))
        self.hotkey_manager.register_callback('switch_to_amharic', lambda: self.switch_mode(InputMode.AMHARIC))
        self.hotkey_manager.register_callback('switch_to_latin', lambda: self.switch_mode(InputMode.LATIN))
        self.hotkey_manager.register_callback('cancel_buffer', self.cancel_buffer)
    
    def switch_mode(self, mode: InputMode):
        """Switch to a different input mode."""
        self.current_mode = mode
        self.status.update_mode(mode)
        
        # Reset composer when switching modes
        self.composer.reset()
    
    def toggle_active(self):
        """Toggle the keyboard active/inactive state."""
        self.active = not self.active
        self.status.toggle_active()
        
        # Reset composer when toggling
        if self.active:
            self.composer.reset()
    
    def cancel_buffer(self):
        """Cancel current input buffer."""
        if self.composer.buffer:
            self.composer.reset()
            self.status.show_status("Buffer cleared")
    
    def setup_keyboard_hook(self):
        """Set up system-wide keyboard hook."""
        try:
            import keyboard
            
            def on_key_event(event):
                """Handle keyboard events."""
                # Only process key down events
                if event.event_type != keyboard.KEY_DOWN:
                    return True
                
                # Skip if not active or in Latin mode
                if not self.active or self.current_mode == InputMode.LATIN:
                    return True
                
                # Only process single alphabetic characters
                if len(event.name) == 1 and event.name.isalpha():
                    result = self.composer.process_input(event.name)
                    
                    # Show debug info if enabled
                    self.status.show_buffer_state(self.composer.buffer, result.suggestions)
                    
                    # Handle the result
                    if result.output:
                        # Output the Geez character
                        keyboard.write(result.output)
                        
                        # Suppress the original keystroke
                        return False
                    
                    elif result.consume_input > 0:
                        # Suppress keystroke but no output (waiting for more input)
                        return False
                
                # Let everything else through
                return True
            
            # Set up the hook
            keyboard.hook(on_key_event, suppress=False)
            self.keyboard_hook_active = True
            
            # Set up hotkeys
            hotkeys_setup = self.hotkey_manager.setup_hotkeys()
            
            if self.config.debug_mode:
                print(f"Keyboard hook active: {self.keyboard_hook_active}")
                print(f"Hotkeys setup: {hotkeys_setup}")
            
            return True
            
        except ImportError:
            self.status.show_status("Keyboard library not available. Please install required dependencies.")
            return False
        except Exception as e:
            self.status.show_status(f"Error setting up keyboard hook: {e}")
            return False
    
    def cleanup(self):
        """Clean up resources."""
        self.hotkey_manager.cleanup()
        self.keyboard_hook_active = False
    
    def get_status(self) -> dict:
        """Get current status information."""
        composer_stats = self.composer.get_stats()
        
        return {
            'active': self.active,
            'current_mode': self.current_mode.value,
            'keyboard_hook_active': self.keyboard_hook_active,
            'current_buffer': self.composer.buffer,
            'composition_strategy': self.config.composition_strategy.value,
            'registered_hotkeys': len(self.hotkey_manager.registered_hotkeys),
            'composer_stats': composer_stats
        }

## Application Functions

High-level functions for starting and managing the keyboard application.

In [ ]:
#| export
def create_default_config(mode: str = "tigrinya", 
                         strategy: str = "adaptive",
                         debug: bool = False) -> KeyboardConfig:
    """Create a default configuration."""
    # Convert string parameters to enums
    try:
        input_mode = InputMode(mode.lower())
    except ValueError:
        print(f"Invalid mode: {mode}. Using Tigrinya.")
        input_mode = InputMode.TIGRINYA
    
    try:
        comp_strategy = CompositionStrategy(strategy.lower())
    except ValueError:
        print(f"Invalid strategy: {strategy}. Using Adaptive.")
        comp_strategy = CompositionStrategy.ADAPTIVE
    
    return KeyboardConfig(
        default_mode=input_mode,
        composition_strategy=comp_strategy,
        debug_mode=debug
    )

def start_keyboard(mode: str = "tigrinya", 
                  strategy: str = "adaptive",
                  debug: bool = False) -> SystemInputHandler:
    """Start the Geez keyboard with specified configuration."""
    
    # Create configuration
    config = create_default_config(mode, strategy, debug)
    
    # Create and setup input handler
    handler = SystemInputHandler(config)
    
    # Setup keyboard hooks
    success = handler.setup_keyboard_hook()
    
    if success:
        handler.status.show_status(
            f"GzKeyboard started in {mode} mode with {strategy} strategy"
        )
        
        if debug:
            status = handler.get_status()
            print(f"Status: {status}")
        
        print("\nHotkeys:")
        print(f"  {config.toggle_hotkey}: Toggle keyboard on/off")
        print(f"  {config.tigrinya_hotkey}: Switch to Tigrinya")
        print(f"  {config.amharic_hotkey}: Switch to Amharic")
        print(f"  {config.latin_hotkey}: Switch to Latin")
        print(f"  {config.cancel_hotkey}: Clear buffer")
        print("\nPress Ctrl+C to exit")
    else:
        print("Failed to start keyboard. Check dependencies and permissions.")
    
    return handler

def stop_keyboard(handler: SystemInputHandler):
    """Stop the keyboard and clean up resources."""
    handler.cleanup()
    handler.status.show_status("GzKeyboard stopped")

## Command Line Interface

Command-line interface for running the keyboard application.

In [ ]:
#| export
def main():
    """Command-line entry point for the Geez Keyboard."""
    parser = argparse.ArgumentParser(
        description="GzKeyboard - Advanced Geez keyboard for typing in Tigrinya and Amharic",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Examples:
  python -m gzkeyboard.system                    # Start with default settings
  python -m gzkeyboard.system --mode amharic     # Start in Amharic mode
  python -m gzkeyboard.system --strategy eager   # Use eager composition
  python -m gzkeyboard.system --test             # Run interactive tests
  python -m gzkeyboard.system --debug            # Enable debug mode
"""
    )
    
    parser.add_argument(
        "--mode", 
        choices=["tigrinya", "amharic", "latin"],
        default="tigrinya",
        help="Input mode (default: tigrinya)"
    )
    
    parser.add_argument(
        "--strategy",
        choices=["eager", "lazy", "predictive", "adaptive"],
        default="adaptive",
        help="Composition strategy (default: adaptive)"
    )
    
    parser.add_argument(
        "--test",
        action="store_true",
        help="Run interactive composition tests"
    )
    
    parser.add_argument(
        "--debug",
        action="store_true",
        help="Enable debug mode with detailed logging"
    )
    
    args = parser.parse_args()
    
    if args.test:
        # Run interactive tests
        from gzkeyboard.composition import test_composition_interactive, compare_strategies
        
        print("GzKeyboard Interactive Tests")
        print("============================\n")
        
        # Create test setup
        config = create_default_config(args.mode, args.strategy, args.debug)
        character_store = CharacterStore()
        composer = SyllableComposer(
            character_store,
            strategy=config.composition_strategy,
            timeout=config.base_timeout
        )
        
        # Test common words
        test_words = ['hello', 'selam', 'geez', 'tigrinya', 'amharic']
        
        for word in test_words:
            output = test_composition_interactive(composer, word)
        
        # Compare strategies
        compare_strategies('hello')
        
        print("\nTests completed!")
        return
    
    # Start the keyboard
    handler = start_keyboard(args.mode, args.strategy, args.debug)
    
    if handler.keyboard_hook_active:
        try:
            # Keep the application running
            while True:
                time.sleep(0.1)
        except KeyboardInterrupt:
            print("\nShutting down GzKeyboard...")
            stop_keyboard(handler)
    else:
        print("Failed to start keyboard hooks. Exiting.")

if __name__ == "__main__":
    main()

## Testing the System Integration

Let's test the integrated system components.

In [ ]:
# Test configuration creation
config = create_default_config("tigrinya", "adaptive", debug=True)
test_eq(config.default_mode, InputMode.TIGRINYA)
test_eq(config.composition_strategy, CompositionStrategy.ADAPTIVE)
test_eq(config.debug_mode, True)

print(f"✓ Configuration test passed")
print(f"  Mode: {config.default_mode.value}")
print(f"  Strategy: {config.composition_strategy.value}")
print(f"  Debug: {config.debug_mode}")

In [ ]:
# Test system input handler (without actual keyboard hooks)
config = KeyboardConfig(debug_mode=True)
handler = SystemInputHandler(config)

# Test mode switching
handler.switch_mode(InputMode.AMHARIC)
test_eq(handler.current_mode, InputMode.AMHARIC)

# Test toggle
original_state = handler.active
handler.toggle_active()
test_eq(handler.active, not original_state)

# Test status
status = handler.get_status()
test_eq(status['current_mode'], 'amharic')

print(f"✓ System handler test passed")
print(f"  Current status: {status}")

In [ ]:
# Test the composition integration
config = KeyboardConfig(composition_strategy=CompositionStrategy.EAGER)
handler = SystemInputHandler(config)

# Simulate key processing (without actual keyboard hooks)
result1 = handler.composer.process_input('h')
result2 = handler.composer.process_input('a')

test_eq(result2.output, 'ሃ')

print(f"✓ Composition integration test passed")
print(f"  'ha' → '{result2.output}' with confidence {result2.confidence}")

# Show final status
final_status = handler.get_status()
print(f"  Final composer stats: {final_status['composer_stats']}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()